In [1]:
path_data = '/'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os
from collections import Counter
import pandas as pd

import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

plt.style.use('ggplot')

Torch version: 2.2.1+cu121


In [3]:
data_unfiltered = pd.read_csv("/".join([path_data, "images_v2.csv"]))


In [4]:
len(data_unfiltered)

38479

### Remove subjective topics

In [5]:
subjective_topics = ['Favorite home decorations', 'Favourite item in kitchen', 'Favourite sports clubs', 'How the most loved item is used', 'icons', 'Idols', 'Latest furniture bought', ' looking over the shoulder', 'Most loved item', 'Most loved toy', 'Most played songs on the radio', 'Music idol', 'Next big thing you are planning to buy', 'Playing with most loved toy', 'Thing I dream about having', 'Things I wish I had', 'Using most loved item', 'Youth culture', 'What I wish I could buy']

In [6]:
unique_topics = data_unfiltered['topics'].unique()

In [7]:
Subjective_topic_list = []

for topic in unique_topics:
    topic_ = topic.replace('/ ', ', ')
    topic_split = topic_.split(',')
    for split in topic_split:
        if split in subjective_topics:
            Subjective_topic_list.append(topic)

In [8]:
data = data_unfiltered[~data_unfiltered['topics'].isin(Subjective_topic_list)]
display(data)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
5,5d4bf31dcf0b3a0f3f35983c,Burundi,bi,af,image,assets/5d4bf31dcf0b3a0f3f35983c/5d4bf31dcf0b3a...,Plate of food,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


In [9]:
len(data)

36754

### Split topics

In [10]:
from collections import Counter
list_topics = list(data['topics'])


separate_topics = [t.lower().strip() for topic in list_topics for t in topic.split(",") ]


print(len(separate_topics))
#print(Counter(separate_topics))
set_topics = list(set(separate_topics))
print(len(set_topics))

45692
270


In [86]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set() #### here a set was used in place of list to avoid duplicate where keyword appears twice in a topic
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

270


In [87]:
#sanity checks 

ground_truth_counts = [len(dict_topic2img[i]) for i in dict_topic2img]
print(sum(ground_truth_counts))

45616


In [88]:
topics = dict_topic2img.keys()

In [89]:
len(topics)

270

### Generated Functions

In [51]:
import openai
import re
openai.api_key = " "
from openai import OpenAI
client = OpenAI(api_key = openai.api_key)

In [53]:
def get_definition(obj):
    '''gets functional definition of one object from gpt4'''
    
    response = client.chat.completions.create(
    messages=[{
        "role": "user",
        "content": f"A bed is an object that provides a place to sleep. Following the above example, In one short sentence, state what function the object '{obj}' does.",
    }],
    model="gpt-4o",)
    
    return response.choices[0].message.content
    

In [54]:
def define_object_list(object_list):
    '''gets functional definition of objects in object list from gpt4'''
    
    function_list = []
    for obj in tqdm.tqdm(object_list):
        function = get_definition(obj)
        function_list.append(function)
        
    return function_list

In [59]:
ob_topics = list(topics)

In [60]:
function_list = define_object_list(ob_topics)

100%|██████████| 270/270 [02:27<00:00,  1.83it/s]


In [136]:
function_list

['Family snapshots capture and preserve memories of shared moments.',
 'Cutlery is a set of utensils used for eating and serving food.',
 'A family provides support and nurturing.',
 'A dining table is an object that provides a place to eat dinner.',
 'A plate of food provides nourishment and sustenance.',
 'A table with food provides a place to eat.',
 'A water dispenser is an object that provides a way to access drinking water.',
 'A stove/hob is an object that provides a surface for cooking food.',
 'A light source in the kitchen provides illumination for cooking and other activities.',
 'Drinking water hydrates and sustains bodily functions.',
 'A kitchen sink is an object that provides a place to wash dishes and hands.',
 'A bathroom provides a private space for personal hygiene activities.',
 'A backyard provides an outdoor space for recreation and relaxation.',
 'A front door provides entry and exit to a building.',
 'A home is an object that provides a place to live.',
 'A sour

In [62]:
def get_question(definition):
    '''gets functional definition of one object from gpt4'''
    
    response = client.chat.completions.create(
    messages=[{
        "role": "user",
        "content": f"'A bed is an object that provides a place to sleep - This is a photo of objects that provide a place to sleep.', 'Salt is a substance that enhances the flavor of food. - This is a photo of a substance that enhances the flavor of food.' Following the above examples in quotes (do not repeat the examples in your response), complete the next sentence by adding 'This is a photo of ' appropriately at the beginning of the sentence. '{definition}'",
    }],
    model="gpt-4o",)
    
    return response.choices[0].message.content

In [63]:
def define_question_list(definition_list):
    '''creates questions using functional definition of objects in object list from gpt4'''
    
    question_list = []
    for definition in tqdm.tqdm(definition_list):
        question = get_question(definition)
        question_list.append(question)
        
    return question_list

In [64]:
def clean_list(question_list):
    question_list_clean = []
    for question in tqdm.tqdm(question_list):
        if '-' in question:
            question_list_clean.append(question.split('-')[-1].replace("\'", ""))

        else:
            question_list_clean.append(question.replace("\'", ""))
    return question_list_clean

In [66]:
function_prompts = define_question_list(function_list)

100%|██████████| 270/270 [02:43<00:00,  1.65it/s]


In [67]:
function_prompts[0]

'This is a photo of memories of shared moments.'

In [78]:
len(topics)

270

In [139]:
len(set(function_prompts))

270

In [140]:
topic2english = {}
for topic, prompt in zip(topics, function_prompts):
    topic2english[topic] = prompt

In [122]:
topic2english['get water']

'This is a photo of an object that provides a way to access drinking water.'

In [141]:
english_dict_topic2imgGT = {}

for topic in tqdm.tqdm(topics):
    key = topic2english[topic]
    english_dict_topic2imgGT[key] = dict_topic2img[topic]
print(len(english_dict_topic2imgGT))

100%|██████████| 270/270 [00:00<00:00, 1281065.70it/s]

270


In [144]:
#english_dict_topic2imgGT

In [135]:
f = open("/dict_genfunction_topic2imgGT.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(english_dict_topic2imgGT ,f)

# close file
f.close()